# [Deep-Neural-Networks 실습 : 실전과제문항]

## 0.모듈 불러오기

In [ ]:
''' 기본 모듈 및 시각화 모듈 '''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

''' 데이터 전처리 모듈 '''
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

'''Neural Network을 위한 딥러닝 모듈'''
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

''' 결과 평가용 모듈 '''
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

! git clone https://github.com/hansam95/LG_time_series_day08.git

## 1. 분석데이터 : Fashion-MNIST 의류 이미지

- 이미지 데이터는 픽셀로 이루어져 있으며, 각 픽셀의 밝기(intensity, 0~255)를 통해 이미지를 나타낼 수 있다.
- Fashion-MNIST 데이터는 이런 픽셀 값들을 설명변수(X)로 하여 어떤 의류(Y)에 해당하는 이미지인지 분류하는 문제
- Fashion-MNIST 데이터의 이미지는 가로 28 픽셀, 세로 28 픽셀로 이루어져 있으므로 28 x 28 = 784개의 설명변수로 10개의 클래스를 예측하는 다범주 분류문제
- 해당 데이터에서의 레이블과 클래스는 아래의 표와 같다.
|레이블|클래스|
|---|---|
|0|T-shirt/top(티셔츠)|
|1|Trouser(바지)|
|2|Pullover(스웨터)|
|3|Dress(드레스)|
|4|Coat(코트)|
|5|Sandal(샌들)|
|6|Shirt(셔츠)|
|7|Sneaker(운동화)|
|8|Bag(가방)|
|9|Ankel boot(발목부츠)|

### 데이터 불러오기

In [ ]:
data = pd.read_csv('''Answer''')

## 1.1 탐색적 데이터 분석

### 데이터 확인

In [ ]:
## 불러온 데이터의 shape 확인
print('Data shape: {}'.format(data.'''Answer'''))
data.head()

### 클래스 비율 확인

In [ ]:
data['Y'].'''Answer'''

### 예시 이미지 출력

In [ ]:
## 1번째 데이터의 예시이미지 출력
image = data.'''Answer'''
plt.imshow(image.reshape(28,28), cmap='gray')
plt.axis('off')
plt.show()

## 1-2. 모델링: Deep Neural Networks (DNN)

### Dataset

In [ ]:
class Fashion_MNISTDataset(Dataset):
    def __init__(self, data_path, split):
        '''데이터 불러오기, 전처리'''

        # 데이터 불러오기 및 설명변수(X)와 반응변수(Y)정의
        data = pd.read_csv(data_path)
        X = '''Answer'''
        y = '''Answer'''

        # 데이터 나누기 (학습, 검증, 테스트)
        train_X, test_X, train_y, test_y = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)
        train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, stratify=train_y, test_size=0.2, random_state=0)

        # 데이터 정규화
        self.scaler = StandardScaler()
        train_X = self.scaler.'''Answer'''
        valid_X = self.scaler.'''Answer'''
        test_X = self.scaler.'''Answer'''

        # 데이터셋 지정
        if split == 'train':
            self.data_X = train_X
            self.data_y = train_y
        elif split == 'valid':
            self.data_X = valid_X
            self.data_y = valid_y
        elif split == 'test':
            self.data_X = test_X
            self.data_y = test_y

        # numpy array -> Tensor 변환
        self.data_X = torch.FloatTensor(self.data_X)
        self.data_y = torch.LongTensor(self.data_y)

    def __len__(self):
        '''데이터셋 개수 정의'''
        return len(self.data_X)
    
    def __getitem__(self, idx):
        '''인덱스를 입력받아 데이터를 출력'''
        X = self.data_X[idx]
        y = self.data_y[idx]
        return X, y

#### DataLoader

In [ ]:
# 학습 데이터
train_dataset = Fashion_MNISTDataset(data_path='/content/test/data/Fashion_MNIST_Example.csv', split='train')
train_loader = DataLoader(dataset = train_dataset, batch_size = 64, shuffle = True)
# 검증 데이터
valid_dataset = Fashion_MNISTDataset(data_path='/content/test/data/Fashion_MNIST_Example.csv', split='valid')
valid_loader = DataLoader(dataset = valid_dataset, batch_size = 64)
# 테스트 데이터
test_dataset = Fashion_MNISTDataset(data_path='/content/test/data/Fashion_MNIST_Example.csv', split='test')
test_loader = DataLoader(dataset = test_dataset, batch_size = 64)

### Model

In [ ]:
class DNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim): 
        super().__init__()
        
        ''' 모델에 필요한 layer 정의 ''' 
               
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim[0]) # input_dim(784) -> hidden_dim1(500)
        self.linear2 = torch.nn.Linear(hidden_dim[0], hidden_dim[1]) # hidden_dim1(500) -> hidden_dim2(500)
        self.linear3 = torch.nn.Linear(hidden_dim[1], hidden_dim[2]) # hidden_dim2(500) -> hidden_dim3(300)
        self.linear4 = torch.nn.Linear(hidden_dim[2], output_dim) # hidden_dim3(300) -> output_dim(10)   
        self.relu = torch.nn.ReLU() # Relu activation function
        
    def forward(self, x):
        
        ''' x를 입력받아 진행할 순서 설정 '''

        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.linear4(x)

        return x

In [ ]:
# 입력 차원수 
input_dim = train_dataset.'''Answer'''
# 모델 선언
model = DNN(input_dim, [500, 300, 100], 10) # 10=class 개수

### 비용함수

In [ ]:
# 비용함수 정의 (분류문제 -> crossentropy 사용)
criterion = torch.nn.CrossEntropyLoss()

### Optimizer

In [ ]:
# 경사하강법의 종류 정의 (adam)
optim = torch.optim.Adam(model.parameters(), lr = 0.0001)

### 학습

In [ ]:
# gpu 사용
device = torch.device('cuda')
# gpu에서 모델 동작
model.to(device)

num_epochs = 100

print("Start Training !")
print('-'*50)

train_loss_total = []
valid_loss_total = []
best_loss = np.inf

for epoch in range(num_epochs):
    
    train_loss = 0
    valid_loss = 0
    
    ''' Training '''
    # 모델을 학습모드로 전환
    model.train()
    # __getitem__에서 반환해주는 형태로 데이터 반환
    for X_data, y_data in train_loader:

        # 데이터를 gpu로
        X_data = X_data.to(device)
        y_data = y_data.to(device)
        
        # 모델의 모든 기울기 값을 0으로 설정
        optim.'''Answer'''
        
        # 데이터를 모델에 입력하여 출력값 계산
        y_pred = '''Answer'''
        
        # 비용함수를 활용하여 오차 계산
        loss = criterion(y_pred, y_data)
        
        # 계산된 오차를 기반으로 기울기 계산
        loss.'''Answer'''
        
        # 계산된 기울기를 바탕으로 모델의 파라미터 업데이트
        optim.'''Answer'''

        train_loss += loss.item()
    
    ''' Validation '''
    # 모델을 평가모드로 전환
    model.eval()
    # 모델을 평가할때는 기울기 계산 불필요
    with torch.no_grad():
        for X_data, y_data in valid_loader:
            X_data = X_data.to(device)
            y_data = y_data.to(device)
            y_pred = '''Answer'''
            loss = criterion(y_pred, y_data)
            valid_loss += loss.item()
    
    # epoch의 loss 계산
    train_loss_epoch = train_loss/len(train_loader)
    valid_loss_epoch = valid_loss/len(valid_loader)

    print(f'{epoch+1} epoch | Train loss: {train_loss_epoch:.3f}, Valid loss: {valid_loss_epoch:.3f}')
    
    # validation loss 기준으로 best epoch 계산 (Early stopping)
    if valid_loss_epoch < best_loss:
        # best loss 업데이트
        best_loss = valid_loss_epoch
        # 최적의 epoch 수와 모델 저장하기
        best_epoch = epoch
        torch.save(model.state_dict(), './best_model_dnn_fashion_MNIST.pt')
    
    train_loss_total.append(train_loss_epoch)
    valid_loss_total.append(valid_loss_epoch)
    
print('-'*50)
print(f'Finished Training ! Best Epoch is epoch {best_epoch+1}')

### 학습 상태 확인 (learning curve)

In [ ]:
plt.figure(figsize=(20,10))

# 학습 및 검증 로스 변동 관찰하기
plt.plot(train_loss_total,label='Train Loss')
plt.plot(valid_loss_total, label='Validation Loss')
# 최적의 모델이 저장된 곳 표시
plt.axvline(x = best_epoch, color='red', label='Best Epoch')
plt.legend(fontsize=20)
plt.title("Learning Curve of trained model", fontsize=18)
plt.show()

## 1-3. Deep Neural Networks 모델 성능 평가

In [ ]:
# 모델 선언
model = DNN(input_dim, [500, 300, 100], 10)

# 최적의 모델 불러오기
ckpt = torch.'''Answer'''

# 모델 파라미터 불러오기
model.load_state_dict(ckpt)

### 학습된 DNN 결과 확인 및 성능 평가: Training Data

In [ ]:
# model을 evaluation 모드로 변경
pred_list = []
true_list = []

model.eval()
with torch.no_grad():
    for X_data, y_data in train_loader:
        # 모델의 출력값 계산
        y_logit = model(X_data)
        # 출력값을 최대로하는 인덱스(class) 저장
        y_pred = torch.argmax(y_logit, dim=1)
        # 예측값과 실제값 저장
        pred_list.extend(y_pred.detach().numpy())
        true_list.extend(y_data.detach().numpy())

In [ ]:
# confusion matrix 계산
cm_train = confusion_matrix(y_true=true_list, y_pred=pred_list)

# confusion matrix 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(data=cm_train, annot=True, fmt='d', annot_kws={'size': 15}, cmap='Blues')
plt.xlabel('Predicted', size=20)
plt.ylabel('True', size=20)
plt.show()

# 평가지표 계산
train_acc = accuracy_score(true_list, pred_list)
train_rec = recall_score(true_list, pred_list, average='macro')
train_prec = precision_score(true_list, pred_list, average='macro')
train_f1 = f1_score(true_list, pred_list, average='macro')

print('Train Accuracy   : {:.3f}'.format(train_acc))
print('Train Sensitivity: {:.3f}'.format(train_rec))
print('Train Precision  : {:.3f}'.format(train_prec))
print('Train F1 Score   : {:.3f}'.format(train_f1))

### 학습된 DNN 결과 확인 및 성능 평가: Testing Data

In [ ]:
# model을 evaluation 모드로 변경
pred_list = []
true_list = []

model.eval()
with torch.no_grad():
    for X_data, y_data in test_loader:
        # 모델의 출력값 계산
        y_logit = model(X_data)
        # 출력값을 최대로하는 인덱스(class) 저장
        y_pred = torch.argmax(y_logit, dim=1)
        # 예측값과 실제값 저장
        pred_list.extend(y_pred.detach().numpy())
        true_list.extend(y_data.detach().numpy())

In [ ]:
# confusion matrix 계산
cm_train = confusion_matrix(y_true=true_list, y_pred=pred_list)

# confusion matrix 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(data=cm_train, annot=True, fmt='d', annot_kws={'size': 15}, cmap='Blues')
plt.xlabel('Predicted', size=20)
plt.ylabel('True', size=20)
plt.show()

# 평가지표 계산
test_acc = accuracy_score(true_list, pred_list)
test_rec = recall_score(true_list, pred_list, average='macro')
test_prec = precision_score(true_list, pred_list, average='macro')
test_f1 = f1_score(true_list, pred_list, average='macro')

print('Test Accuracy   : {:.3f}'.format(test_acc))
print('Test Sensitivity: {:.3f}'.format(test_rec))
print('Test Precision  : {:.3f}'.format(test_prec))
print('Test F1 Score   : {:.3f}'.format(test_f1))

## 1-4. 실제 데이터 이미지로 확인

|레이블|클래스|
|---|---|
|0|T-shirt/top(티셔츠)|
|1|Trouser(바지)|
|2|Pullover(스웨터)|
|3|Dress(드레스)|
|4|Coat(코트)|
|5|Sandal(샌들)|
|6|Shirt(셔츠)|
|7|Sneaker(운동화)|
|8|Bag(가방)|
|9|Ankel boot(발목부츠)|

In [ ]:
# test data에서 일부 이미지 선택
test_data = test_dataset.data_X[:9]
y_true = test_dataset.data_y[:9]

# 모델의 출력값 계산
y_logit = model(test_data)

# 출력값을 최대로하는 인덱스(class) 저장
y_pred = '''Answer'''

# scaler 불러오기
scaler = test_dataset.scaler

# 정규화 제거 (-1 ~ 1 -> 0 ~ 255)
test_data = scaler.'''Answer'''

In [ ]:
# 예측값 출력
print('예측값 출력')
print(y_pred.reshape(3,3))

print('실제값 출력')
print(y_true.reshape(3,3))

# test data 시각화
fig ,axes = plt.subplots(3,3,figsize=(10,10))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(test_data[i].reshape(28,28), cmap='gray')
    ax.axis('off')
plt.show()